<a href="https://colab.research.google.com/github/jodog0412/US-STOCKS-INVESTMENT-ASSISTANCE/blob/master/stock_investment_assistance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Environment&Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cd /content/drive/MyDrive/Colab Notebooks/US-STOCKS-INVESTMENT-ASSISTANCE

In [1]:
!pip install yfinance --upgrade --no-cache-dir
!pip install finance-datareader --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 KB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 98.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 KB 126.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 102.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 KB 125.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1
  Attempting uninstall: html5lib
    Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
 

In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
yf.pdr_override()
import FinanceDataReader as fdr
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

In [119]:
startdate='2022-01-01'
enddate='2023-01-23'

#2. Analyze Portfolio

## 1) Load spreadsheet

In [ ]:
from google.colab import auth
auth.authenticate_user()
from google.auth import default
import gspread
creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
sheet=gc.open('US_STOCK').sheet1

## 2) Analyze portfolio

In [ ]:
class portfolio:
    def __init__(self,sheet):
        data=pd.DataFrame(sheet.get_all_values())
        pf=pd.DataFrame(data=data.values,
                        columns=["TICKER","TAG","AVER_PRICE","STOCK_NUM","STOCK_QUANTITY"]).sort_values(by="TICKER")
        self.pf=pf.sort_values("STOCK_QUANTITY",ascending=False)
        
    def plot(self):
        data=self.pf
        pf_plt=plt.pie(data["STOCK_QUANTITY"],labels=data.loc[:,"TICKER"],
                       autopct='%.1f%%',startangle=90)
        return(plt.show(pf_plt))
    
    def calcProfit(self,date):
        ticker=self.pf.loc[:,"TICKER"].values
        data=yf.download(list(ticker),start=startdate,end=enddate,progress=False)
        last_price=data['Adj Close'].iloc[-1]
        profit=(last_price/(self.pf)['AVER_PRICE'].values.astype(float)-1)*100
        profit=profit.sort_values(ascending=False)
        return(profit)

In [ ]:
sheetData=portfolio(sheet)
sheetData.plot()

NameError: ignored

## 3. Search Tickers

## 1) Load Tickers

In [99]:
nyse, ndq = fdr.StockListing('NYSE'), fdr.StockListing('NASDAQ')

100%|██████████| 4432/4432 [00:33<00:00, 133.90it/s]


##2) Load Sectors

In [105]:
sectors=list(set(nyse['Industry'].values))
pd.Series(sectors).sort_values().values

array(['가정용품', '개인, 가정 용품 및 서비스', '건설 및 엔지니어링', '건설장비 및 대형차, 기차, 배',
       '건축 자재', '교육서비스', '기타 금융업', '다양한 금융서비스', '다양한 산업재 도매업', '다양한 소매업',
       '레저 상품', '메탈&마이닝', '미디어 및 출판', '반도체 및 반도체 장비', '보험', '복합 기업',
       '복합 유틸리티', '부동산 운영 관리', '비즈니스교육', '사무기기', '생명과학 및 메디컬 리서치', '석탄',
       '섬유 및 의류', '소프트웨어 및 IT서비스', '수도 유틸리티', '식품 및 담배', '식품 및 약품소매',
       '오일 및 가스', '오일 및 가스 장비 및 서비스', '용기 및 포장', '우라늄', '우주항공 및 국방',
       '운송 인프라', '운송업', '은행', '음료', '자동차 및 자동차 부품', '재생에너지', '전기 유틸리티',
       '전문 및 상업 서비스', '전문품 상점', '전자 장비 및 부품', '정부활동', '제약', '종이 및 임산물',
       '주택 및 상업용 리츠', '주택건설', '천연가스 유틸리티', '커뮤니케이션 및 네트워킹',
       '컴퓨터, 전화 및 가전제품', '통신 서비스', '통합 하드웨어 및 소프트웨어', '투자은행 및 서비스', '핀테크',
       '학교', '헬스케어 업체 및 서비스', '헬스케어 장비 및 용품', '호텔 및 엔터테인먼트',
       '화물 및 물류 서비스', '화학', nan], dtype=object)

## 3) Search tickers

In [106]:
class tickerSearch:
    def __init__(self,index,sector):
        if index=='NYSE':
            data = dict(list(nyse.groupby('Industry')))
        if index=='NASDAQ':
            data = dict(list(ndq.groupby('Industry')))
        tickers=list(data[sector]['Symbol'].values)
        self.tickerFilt=[ticker for ticker in tqdm(tickers) if yf.download(ticker,progress=False).empty!=True]

    def download(self):
        tickers=self.tickerFilt
        data=yf.download(tickers,start=startdate,end=enddate)
        result=(data['Adj Close'].iloc[-1]/data['Adj Close'].iloc[0]-1).sort_values(ascending=False).dropna()
        return result

In [107]:
tickerInit=tickerSearch('NYSE','컴퓨터, 전화 및 가전제품')
searchResult=tickerSearch.download(tickerInit)

100%|██████████| 13/13 [00:04<00:00,  2.75it/s]

[                       0%                       ]

[*********************100%***********************]  13 of 13 completed


In [108]:
filterN=10
searchTickers=searchResult.index[:filterN]

## 4. Compare Financials

In [112]:
def financialCompare(command):
    if command=='SEARCH':
        tickers=tickerDict['SEARCH']
    else:
        tickers=tickerDict['WATCH']
    def getFinancial(ticker):
        def return_(info, key):
            try: return info[key]
            except KeyError: return "NaN"
        info=yf.Ticker(ticker).info
        infoCustom=[return_(info,"trailingPE"), return_(info,"forwardPE"), return_(info,"priceToBook"),
                    return_(info,"pegRatio"), return_(info,"returnOnEquity"),return_(info,"shortPercentOfFloat"),
                    return_(info,"quickRatio")]
        return infoCustom

    financial=[getFinancial(ticker) for ticker in tqdm(tickers)]
    df=pd.DataFrame(data=financial,
                    index=tickers,
                    columns=['PER', 'FPER', 'PBR',
                             'PEGR', 'ROE', 'short%', 'qRatio'])
    df=df.sort_values('FPER',ascending=False)
    return df

  0%|          | 0/10 [00:30<?, ?it/s]


In [113]:
tickerDict={
    'SEARCH':searchTickers,
    'WATCH':['CLFD','ON','ONTO','RELL','META','EQNR','STEM','ANDE','POST','UFPT']
}

In [ ]:
financialCompare('SEARCH')

In [120]:
data=yf.download(tickerDict['WATCH'],start=startdate,end=enddate)
result=(data['Adj Close'].iloc[-1]/data['Adj Close'].iloc[0]-1).sort_values(ascending=False).dropna()
result

[*********************100%***********************]  10 of 10 completed


RELL    0.969671
UFPT    0.671279
EQNR    0.266211
POST    0.228932
ON     -0.055722
ANDE   -0.069114
CLFD   -0.086497
ONTO   -0.265129
STEM   -0.468815
META   -0.588320
dtype: float64

#5. Visualize Financials

In [157]:
def revenueGrowthTable(tickers):
    revDict={}
    for ticker in tqdm(tickers):
        data=yf.Ticker(ticker).earnings
        revGrowth=[(data.iloc[i+1,0]-data.iloc[i,0])/data.iloc[i,0]*100 for i in range(3)]
        revDict[ticker]=revGrowth
    result=pd.DataFrame(data=revDict)
    return result

In [158]:
revenueGrowthTable(tickerDict['WATCH'])

100%|██████████| 10/10 [00:10<00:00,  1.05s/it]


,CLFD,ON,ONTO,RELL,META,EQNR,STEM,ANDE,POST,UFPT
0,9.455983,-6.131024,11.728954,-6.452968,26.610910,-19.915729,149.601820,162.799642,-17.075918,4.161613
1,51.227505,-4.764494,81.923268,13.495362,21.596390,-27.273450,106.853920,0.766776,5.724899,-9.581563
2,92.450002,28.254995,41.761846,26.949140,37.182574,93.963237,250.816647,56.387406,17.477463,15.022885


In [159]:
def cashflowTable(tickers):
    flowDict={}
    for ticker in tqdm(tickers):
        balance=yf.Ticker(ticker).cashflow.loc[['Operating Cash Flow','Financing Cash Flow','Investing Cash Flow']]
        balance.index=['Operating','Financing','Investing']
        balance=balance.transpose().sort_index()
        flowDict[ticker]=balance
    return flowDict

In [156]:
cashflowTable(tickerDict['WATCH'])

100%|██████████| 10/10 [00:11<00:00,  1.20s/it]


{'CLFD':              Operating   Financing   Investing
 2019-09-30  14732653.0   -236670.0 -12962039.0
 2020-09-30   6655773.0   -246730.0    -41128.0
 2021-09-30  10902839.0   -536435.0 -13599759.0
 2022-09-30   1001000.0  10655000.0  -8197000.0,
 'ON':                Operating    Financing     Investing
 2019-12-31   694700000.0  623100000.0 -1510900000.0
 2020-12-31   884300000.0 -244000000.0  -453600000.0
 2021-12-31  1782000000.0 -569400000.0  -915100000.0,
 'ONTO':               Operating   Financing    Investing
 2019-12-31   18146000.0  -4198000.0    4104000.0
 2020-12-31  105984000.0 -53702000.0  -48599000.0
 2021-12-31  175281000.0   2670000.0 -141793000.0,
 'RELL':             Operating  Financing   Investing
 2019-05-31 -2563000.0 -2817000.0 -11874000.0
 2020-05-31  1923000.0 -3208000.0  -9776000.0
 2021-05-31   832000.0 -3014000.0  13368000.0
 2022-05-31  1911000.0  -352000.0  -8120000.0,
 'META':                 Operating      Financing      Investing
 2019-12-31  363140